In [1]:
using Revise

In [2]:
using ITensors
using Test
import CTMRG

In [93]:
ITensors.disable_warn_order()

14

In [7]:
function ising_transfer_element_2D(i1::Int,i2::Int,j1::Int,j2::Int,
    β::Float64, J::Float64)
    σ1 = 2*i1 -1 #up spin
    σ2 = 2*i2 -1 #down spin
    s1 = 2*j1 -1 #left spin
    s2 = 2*j2 -1 #right spin
    return exp(β*J*(σ1*s1 + s1*σ2 + σ2*s2 + s2*σ1))
end

ising_transfer_element_2D (generic function with 1 method)

In [8]:
β = 1.0
J = 1.0
ising_transfer_element_2D(1, 1, 1, 1, β, J)

54.598150033144236

In [9]:
@testset "ising_transfer_element_2Dのテスト" begin
    β = 1.0
    J = 1.0
    @test ising_transfer_element_2D(1, 1, 1, 1, β, J) ≈ exp(β*J*4) #全てのスピンがup
    @test ising_transfer_element_2D(0, 0, 0, 0, β, J) ≈ exp(β*J*4) #全てのスピンがdown
    @test ising_transfer_element_2D(1, 0, 0, 0, β, J) ≈ 1.0
    @test ising_transfer_element_2D(1, 1, 0, 0, β, J) ≈ exp(-β*J*4)
end

Test Summary:                 | Pass  Total  Time
ising_transfer_element_2Dのテスト |    4      4  0.2s


Test.DefaultTestSet("ising_transfer_element_2Dのテスト", Any[], 4, false, false, true, 1.754612956312843e9, 1.754612956474644e9, false, "In[9]")

In [10]:
#2Dイジングの転送行列を定義する関数
function ising_transfer_tensor_2D(β::Float64, J::Float64,
    σ1::Index, σ2::Index, s1::Index, s2::Index)
    T = ITensor(σ1, σ2, s1, s2)
    for i1 in 0:1, i2 in 0:1,
        j1 in 0:1, j2 in 0:1
        T[σ1=>i1+1, σ2=>i2+1, s1=>j1+1, s2=>j2+1] =
            ising_transfer_element_2D(i1, i2, j1, j2, β, J)
    end
    return T
end

ising_transfer_tensor_2D (generic function with 1 method)

In [11]:
@testset "ising_transfer_tensor_2Dのテスト" begin
    σ1 = Index(2, "σ1,m=1")
    σ2 = Index(2, "σ2,m=1")
    s1 = Index(2, "s1,n=1")
    s2 = Index(2, "s2,n=1")
    β = 1.0
    J = 1.0
    T = ising_transfer_tensor_2D(β, J, σ1, σ2, s1, s2)
    @test T[σ1=>1, σ2=>1, s1=>1, s2=>1] ≈ exp(β*J*4) #全てのスピンがup
    @test T[σ1=>2, σ2=>2, s1=>2, s2=>2] ≈ exp(β*J*4) #全てのスピンがdown
    @test T[σ1=>1, σ2=>2, s1=>2, s2=>2] ≈ 1.0
    @test T[σ1=>1, σ2=>1, s1=>2, s2=>2] ≈ exp(-β*J*4)
end

Test Summary:                | Pass  Total  Time
ising_transfer_tensor_2Dのテスト |    4      4  0.6s


Test.DefaultTestSet("ising_transfer_tensor_2Dのテスト", Any[], 4, false, false, true, 1.754612959557284e9, 1.754612960124202e9, false, "In[11]")

In [12]:
#境界条件を作る関数
function make_boundary_tensors(L::Int, ind_list::Vector{<:Index})#Indexの全ての方を受け入れる
    l = length(ind_list)
    @assert l == 2*L "Length of ind_list must match L"
    boundary_list = Vector{ITensor}(undef, l)
    for (i, ind) in enumerate(ind_list)
        boundary_list[i] = CTMRG.fixend(ind)
    end
    return boundary_list
end

make_boundary_tensors (generic function with 1 method)

In [13]:
ind_list = [Index(2, "s,m=$i") for i in 1:2]
boundary_list = make_boundary_tensors(1, ind_list)
@testset "make_boundary_tensorsのテスト" begin
    ind_list = [Index(2, "s,m=$i") for i in 1:2]
    boundary_list = make_boundary_tensors(1, ind_list)
    @test length(boundary_list) == 2
    @test all(idx -> dim(idx) == 2, boundary_list)
end

Test Summary:             | Pass  Total  Time
make_boundary_tensorsのテスト |    2      2  0.0s


Test.DefaultTestSet("make_boundary_tensorsのテスト", Any[], 2, false, false, true, 1.754612962665806e9, 1.754612962691559e9, false, "In[13]")

In [14]:
#内部のテンソルを作る関数
function make_inner_tensors(L::Int, β::Float64, J::Float64,
    sind_list::Array{<:Index},σind_list::Array{<:Index})
    length = 2*L
    T_list = Array{ITensor}(undef, length, length)
    for i in 1:length, j in 1:length
        T_list[i, j] = ising_transfer_tensor_2D(β, J,
            σind_list[j,i], σind_list[j+1,i],
            sind_list[j,i], sind_list[j,i+1])
    end
    return T_list
end    


make_inner_tensors (generic function with 1 method)

In [15]:
@testset "make_inner_tensorsのテスト" begin
    σ = CTMRG.make_2D_index_σtags(1)
    s = CTMRG.make_2D_index_stags(1)
    T_list = make_inner_tensors(1, β, J, s, σ)
    for i in 1:size(T_list, 1), j in 1:size(T_list, 2)
        @test T_list[i,j][σ[j,i]=>1, σ[j+1,i]=>1, s[j,i]=>1, s[j,i+1]=>1] ≈ exp(β*J*4) #全てのスピンがup
        @test T_list[i,j][σ[j,i]=>2, σ[j+1,i]=>2, s[j,i]=>2, s[j,i+1]=>2] ≈ exp(β*J*4) #全てのスピンがdown
        @test T_list[i,j][σ[j,i]=>1, σ[j+1,i]=>2, s[j,i]=>2, s[j,i+1]=>2] ≈ 1.0
        @test T_list[i,j][σ[j,i]=>1, σ[j+1,i]=>1, s[j,i]=>2, s[j,i+1]=>2] ≈ exp(-β*J*4)
    end
    @test size(T_list) == (2, 2) # 1x1の格子なので2x2のテンソル
end

Test Summary:          | Pass  Total  Time
make_inner_tensorsのテスト |   17     17  0.0s


Test.DefaultTestSet("make_inner_tensorsのテスト", Any[], 17, false, false, true, 1.754612966225057e9, 1.754612966227216e9, false, "In[15]")

In [12]:
#tagの一部抜き出しの練習
σ_list = CTMRG.make_2D_index_σtags(1)
@show σ_list[1,:]
@show σ_list[3,:]
s_list = CTMRG.make_2D_index_stags(1)
@show s_list[:,1]
@show s_list[:,3]
;


σ_list[1, :] = Index[(dim=2|id=216|"i=1,j=1,σ"), (dim=2|id=31|"i=2,j=1,σ")]
σ_list[3, :] = Index[(dim=2|id=441|"i=1,j=3,σ"), (dim=2|id=921|"i=2,j=3,σ")]
s_list[:, 1] = Index[(dim=2|id=372|"i=1,j=1,s"), (dim=2|id=612|"i=1,j=2,s")]
s_list[:, 3] = Index[(dim=2|id=958|"i=3,j=1,s"), (dim=2|id=265|"i=3,j=2,s")]


In [16]:
#2Dの分配関数を作る関数
function setup_tensors_2D(L::Int, β::Float64, J::Float64)
    σ_list = CTMRG.make_2D_index_σtags(L)
    s_list = CTMRG.make_2D_index_stags(L)
    T_list = make_inner_tensors(L, β, J, s_list, σ_list)
    Dup_list = make_boundary_tensors(L, σ_list[2*L+1,:]) #上の境界
    Ddown_list = make_boundary_tensors(L, σ_list[1,:]) #下の境界
    L_list = make_boundary_tensors(L, s_list[:,1]) #左の境界
    R_list = make_boundary_tensors(L, s_list[:,2*L+1]) #右の境界
    return T_list, Dup_list, Ddown_list, L_list, R_list
end


setup_tensors_2D (generic function with 1 method)

In [17]:
L = 2
T_list, Dup_list, Ddown_list, L_list, R_list = setup_tensors_2D(L, β, J)
;

In [18]:

T_list, Dup_list, Ddown_list, L_list, R_list = setup_tensors_2D(1, β, J)
result = ITensor(true)
#@show size(T, 1)
#@show size(T, 2)
T = ITensor(true)
for T in T_list
    result *= T
end
result
#@show Z = only(Dup_list * Ddown_list * L_list * R_list * result)
# 上端
for T in Dup_list
    result *= T
end

# 左端
for T in L_list
    result *= T
end

# 右端
for T in R_list
    result *= T
end

# 下端
for T in Ddown_list
    result *= T
end
@show only(result)
;

only(result) = 8.898259745056173e6


In [19]:
function partition_function_2D(L::Int, β::Float64, J::Float64)
    T_list, Dup_list, Ddown_list, L_list, R_list = setup_tensors_2D(L, β, J)
    result = ITensor(true)
    # 内部テンソルをすべて掛ける
    for T in T_list
        result *= T
    end
    # 上端
    for T in Dup_list
        result *= T
    end
    # 左端
    for T in L_list
        result *= T
    end
    # 右端
    for T in R_list
        result *= T
    end
    # 下端
    for T in Ddown_list
        result *= T
    end
    return only(result)
end

partition_function_2D (generic function with 1 method)

In [ ]:
L = 1
half_ind = 1
β = 1.0
J = 1.0
T_list, Dup_list, Ddown_list, L_list, R_list = setup_tensors_2D(L, β, J)
L_list_up = L_list[1:half_ind]
L_list_down = L_list[1:half_ind]
R_list_up = R_list[1:half_ind]
R_list_down = R_list[half_ind+1:end]
T_list_up = T_list[1:half_ind, 1:half_ind]
T_list_down = T_list[half_ind+1:end, half_ind+1:end]

# 上端の境界条件を
;

In [21]:
function contruct_with_boundary(L::Int, β::Float64, J::Float64)
    T_list, Dup_list, Ddown_list, L_list, R_list = setup_tensors_2D(L, β, J)
    T1_list = copy(T_list) #テンソルのコピーを作成
    # 上端の境界条件を適用
    for (i, Ddown) in enumerate(Ddown_list)
        T1_list[i,1] = T_list[i,1] * Ddown
    end
    # 左端の境界条件を適用
    for (i, L) in enumerate(L_list)
        ord = order(T1_list[1,i])
        if ord == 0
            T1_list[1,i] = T_list[1,i] * L
        else
            T1_list[1,i] = T1_list[1,i] * L
        end
    end
    # 右端の境界条件を適用
    for (i, Dup) in enumerate(Dup_list)
        ord = order(T1_list[i,2*L])
        if ord == 0
            T1_list[i,2*L] = T_list[i,2*L] * Dup
        else
            T1_list[i,2*L] = T1_list[i,2*L] * Dup
        end
    end
    # 下端の境界条件を適用
    for (i, R) in enumerate(R_list)
        ord = order(T1_list[2*L,i])
        if ord == 0
            T1_list[2*L,i] = T_list[2*L,i] * R
        else
            T1_list[2*L,i] = T1_list[2*L,i] * R
        end
    end
    return T_list, T1_list
end


contruct_with_boundary (generic function with 1 method)

In [22]:
@testset "contract_with_boundaryのテスト" begin
    L = 4
    β = 1.0
    J = 1.0

    T_list, T1_list = contruct_with_boundary(L, β, J)

    @test size(T_list) == size(T1_list) == (2L, 2L)

    for i in 1:2L, j in 1:2L
        inds_orig = Set(inds(T_list[i,j]))
        inds_new  = Set(inds(T1_list[i,j]))

        is_boundary = (i == 1 || i == 2L || j == 1 || j == 2L)

        if is_boundary
            @test inds_orig != inds_new  # 境界のテンソルは ind が変化しているはず
        else
            @test inds_orig == inds_new  # 内部のテンソルは変化していないはず
        end
    end
end


Test Summary:              | Pass  Total  Time
contract_with_boundaryのテスト |   65     65  0.1s


Test.DefaultTestSet("contract_with_boundaryのテスト", Any[], 65, false, false, true, 1.754613005633043e9, 1.754613005736237e9, false, "In[22]")

In [96]:
#上側と下側に分けて
L = 5
half_ind = 2
β = 1.0
J = 1.0
T_list, T1_list = contruct_with_boundary(L, β, J)
T1_list_upper = T1_list[1:half_ind,:]
T1_list_lower = T1_list[half_ind+1:end,:]
@show size(T1_list_upper)
@show size(T1_list_lower)
;

size(T1_list_upper) = (2, 10)
size(T1_list_lower) = (8, 10)


In [98]:
#期待値計算
result_upper = ITensor(true)
result_lower = ITensor(true)
for T in T1_list_upper
    result_upper *= T
end
for T in T1_list_lower
    result_lower *= T
end
result_upper
result_lower

ITensor ord=10 (dim=2|id=320|"i=3,j=1,s") (dim=2|id=644|"i=3,j=2,s") (dim=2|id=418|"i=3,j=3,s") (dim=2|id=929|"i=3,j=4,s") (dim=2|id=742|"i=3,j=5,s") (dim=2|id=184|"i=3,j=6,s") (dim=2|id=206|"i=3,j=7,s") (dim=2|id=97|"i=3,j=8,s") (dim=2|id=595|"i=3,j=9,s") (dim=2|id=29|"i=3,j=10,s")
NDTensors.Dense{Float64, Vector{Float64}}

In [99]:
# 例: inds(T) で全Index取得
for (n, ind) in enumerate(inds(result_upper))
    println("Index $n: ", ind)
end

Index 1: (dim=2|id=320|"i=3,j=1,s")
Index 2: (dim=2|id=644|"i=3,j=2,s")
Index 3: (dim=2|id=418|"i=3,j=3,s")
Index 4: (dim=2|id=929|"i=3,j=4,s")
Index 5: (dim=2|id=742|"i=3,j=5,s")
Index 6: (dim=2|id=184|"i=3,j=6,s")
Index 7: (dim=2|id=206|"i=3,j=7,s")
Index 8: (dim=2|id=97|"i=3,j=8,s")
Index 9: (dim=2|id=595|"i=3,j=9,s")
Index 10: (dim=2|id=29|"i=3,j=10,s")


In [100]:
# タグで取り出す場合
@show target = findfirst(ind -> hastags(ind, "i=3,j=2,s"), inds(result_upper))
@show myind = inds(result_upper)[target]
newind = settags(myind, "prev")  # タグを "prev" に変更
T2 = replaceind(result_upper, myind, newind)

target = findfirst((ind->begin
                #= In[100]:2 =#
                hastags(ind, "i=3,j=2,s")
            end), inds(result_upper)) = 2
myind = (inds(result_upper))[target] = (dim=2|id=644|"i=3,j=2,s")


ITensor ord=10 (dim=2|id=320|"i=3,j=1,s") (dim=2|id=644|"prev") (dim=2|id=418|"i=3,j=3,s") (dim=2|id=929|"i=3,j=4,s") (dim=2|id=742|"i=3,j=5,s") (dim=2|id=184|"i=3,j=6,s") (dim=2|id=206|"i=3,j=7,s") (dim=2|id=97|"i=3,j=8,s") (dim=2|id=595|"i=3,j=9,s") (dim=2|id=29|"i=3,j=10,s")
NDTensors.Dense{Float64, Vector{Float64}}

In [ ]:
T1_list_upper = T1_list[1:half_ind,:]
T1_list_lower = T1_list[half_ind+1:end,:]

In [101]:
#targetのtagを貼り替える関数
function replace_tag_ITensor(T::ITensor, tag::String)
    target = findfirst(ind -> hastags(ind, "$tag"), inds(T))
    myind = inds(T)[target]
    newind = prime(myind)
    #newind = settags(myind, "prev") 
    T2 = replaceind(T, myind, newind)
    return T2, newind
end


replace_tag_ITensor (generic function with 1 method)

In [102]:
target = findfirst(ind -> hastags(ind, "i=3,j=2,s"), inds(result_upper))
myind = inds(result_upper)[target]
T2, newind = replace_tag_ITensor(result_upper, "i=3,j=2,s")
σ_tensor = CTMRG.spin_tensor(myind, newind)
σ = T2 * σ_tensor 

ITensor ord=10 (dim=2|id=320|"i=3,j=1,s") (dim=2|id=418|"i=3,j=3,s") (dim=2|id=929|"i=3,j=4,s") (dim=2|id=742|"i=3,j=5,s") (dim=2|id=184|"i=3,j=6,s") (dim=2|id=206|"i=3,j=7,s") (dim=2|id=97|"i=3,j=8,s") (dim=2|id=595|"i=3,j=9,s") (dim=2|id=29|"i=3,j=10,s") (dim=2|id=644|"i=3,j=2,s")
NDTensors.Dense{Float64, Vector{Float64}}

In [106]:
#自発磁化を計算する関数
function calc_spontaneous_magnetization(L::Int, β::Float64, J::Float64
    ,tag::String)
    Z = partition_function_2D(L, β, J)
    T_list, T1_list = contruct_with_boundary(L, β, J)
    # 上側と下側に分ける
    result_upper = ITensor(true)
    result_lower = ITensor(true)
    for T in T1_list[1:L,:]
        result_upper *= T
    end
    for T in T1_list[L+1:end,:]
        result_lower *= T
    end
    target = findfirst(ind -> hastags(ind, "$tag"), inds(result_upper))
    myind = inds(result_upper)[target]
    T2, newind = replace_tag_ITensor(result_upper, "$tag")
    σ_tensor = CTMRG.spin_tensor(myind, newind)
    σ = T2 * σ_tensor 
    return only(result_lower * σ) / Z
end

calc_spontaneous_magnetization (generic function with 1 method)

In [110]:
L = 3
β = 1.0
J = 1.0
calc_spontaneous_magnetization(L, β, J,"i=$(L+1),j=$L,s")

0.9992757522241872

In [111]:
function calc_spin_correlation(L::Int, β::Float64, J::Float64,
    tag1::String, tag2::String)
    Z = partition_function_2D(L, β, J)
    T_list, T1_list = contruct_with_boundary(L, β, J)
    # 上側と下側に分ける
    result_upper = ITensor(true)
    result_lower = ITensor(true)
    for T in T1_list[1:L,:]
        result_upper *= T
    end
    for T in T1_list[L+1:end,:]
        result_lower *= T
    end
    target1 = findfirst(ind -> hastags(ind, "$tag1"), inds(result_upper))
    myind1 = inds(result_upper)[target1]
    target2 = findfirst(ind -> hastags(ind, "$tag2"), inds(result_upper))
    myind2 = inds(result_upper)[target2]
    T2_1, newind1 = replace_tag_ITensor(result_upper, "$tag1")
    T2_2, newind2 = replace_tag_ITensor(T2_1, "$tag2")
    σ_tensor_1 = CTMRG.spin_tensor(myind1, newind1)
    σ_tensor_2 = CTMRG.spin_tensor(myind2, newind2)

    σσ = T2_2 * σ_tensor_1 * σ_tensor_2 
    return only(result_lower * σσ) / Z
end

calc_spin_correlation (generic function with 1 method)

In [115]:
L = 4
β = 1.0
J = 1.0
calc_spin_correlation(L, β, J,"i=$(L+1),j=$L,s","i=$(L+1),j=$(L+1),s")

: 

In [95]:
only(result_lower * σ)/ Z


0.9992758286497886

In [22]:
T_list[1:4, 1:4]

4×4 Matrix{ITensor}:
 ITensor ord=4
Dim 1: (dim=2|id=47|"i=1,j=1,σ")
Dim 2: (dim=2|id=137|"i=1,j=2,σ")
Dim 3: (dim=2|id=130|"i=1,j=1,s")
Dim 4: (dim=2|id=550|"i=2,j=1,s")
Dense{Float64, Vector{Float64}}
 2×2×2×2
[:, :, 1, 1] =
 54.5982  1.0
  1.0     0.0183156

[:, :, 2, 1] =
 1.0  1.0
 1.0  1.0

[:, :, 1, 2] =
 1.0  1.0
 1.0  1.0

[:, :, 2, 2] =
 0.0183156   1.0
 1.0        54.5982   …  ITensor ord=4
Dim 1: (dim=2|id=187|"i=1,j=4,σ")
Dim 2: (dim=2|id=57|"i=1,j=5,σ")
Dim 3: (dim=2|id=591|"i=1,j=4,s")
Dim 4: (dim=2|id=104|"i=2,j=4,s")
Dense{Float64, Vector{Float64}}
 2×2×2×2
[:, :, 1, 1] =
 54.5982  1.0
  1.0     0.0183156

[:, :, 2, 1] =
 1.0  1.0
 1.0  1.0

[:, :, 1, 2] =
 1.0  1.0
 1.0  1.0

[:, :, 2, 2] =
 0.0183156   1.0
 1.0        54.5982
 ITensor ord=4
Dim 1: (dim=2|id=63|"i=2,j=1,σ")
Dim 2: (dim=2|id=445|"i=2,j=2,σ")
Dim 3: (dim=2|id=550|"i=2,j=1,s")
Dim 4: (dim=2|id=895|"i=3,j=1,s")
Dense{Float64, Vector{Float64}}
 2×2×2×2
[:, :, 1, 1] =
 54.5982  1.0
  1.0     0.0183156

[:, 